In [ ]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
import datetime

In [ ]:
def CapitalData(date_str):

    #datestr = '20231002'
    datestr = date_str

    r = requests.get('https://www.twse.com.tw/rwd/zh/fund/T86?date=' + datestr + '&selectType=ALLBUT0999&response=csv')

    # 整理資料，變成表格

    df = pd.read_csv(StringIO(r.text.replace("=", "")), header=1).dropna(how='all', axis=1).dropna(how='any')

    # Remove Column
    df = df.drop(columns='外資自營商買進股數')
    df = df.drop(columns='外資自營商賣出股數')
    df = df.drop(columns='外資自營商買賣超股數')

    field_names = [
    'code',
    'name',
    'ForeignInvestorBuyShares',
    'ForeignInvestorSellShares',
    'ForeignInvestorNetShares',
    'InvestmentTrustBuyShares',
    'InvestmentTrustSellShares',
    'InvestmentTrustNetShares',
    'DealerSelfNetTotalShares',
    'DealerSelfBuyShares',
    'DealerSelfSellShares',
    'DealerSelfNetShares',
    'DealerSelfBuySharesHedge',
    'DealerSelfSellSharesHedge',
    'DealerSelfNetSharesHedge',
    'TotalCapitalQuantity'
    ]

    # Rename the columns of the DataFrame
    df.columns = field_names

    # Turn into numeric
    df[df.columns[2:]] = df[df.columns[2:]].applymap(lambda x: pd.to_numeric(str(x).replace(",", ""), errors='coerce'))

    # Insert Date
    df['Date'] = datestr

    # Convert the 'Date' column to datetime format
    df['Date'] = pd.to_datetime(df['Date']).dt.date

    return df 

In [ ]:
# Define the start date and end date
start_date = datetime.date(2023, 9, 25)
end_date = datetime.date(2023, 10, 3)

# Generate a date range between start_date and end_date
date_range = pd.date_range(start=start_date, end=end_date)

# Filter the date range to keep only weekdays (Monday to Friday)
workdays = [date for date in date_range if date.weekday() < 5]

# Convert the workdays to a list of strings in the format 'YYYY-MM-DD'
workdays_str = [date.strftime('%Y%m%d') for date in workdays]

In [ ]:
import time
from tqdm import tqdm

# Initialize an empty DataFrame to store the concatenated data
all_data = pd.DataFrame()

# Initialize the tqdm progress bar
for date_str in tqdm(workdays_str):
    try:
        data = CapitalData(date_str)
        
        # Concatenate the data to the all_data DataFrame
        all_data = pd.concat([all_data, data], ignore_index=True)
        time.sleep(1)

    except Exception as e:
        pass

print("Finish Collecting!!")

In [ ]:
from google.cloud import bigquery

# Replace this with your service account key file path
service_account_key_path = "StockBiqQueryKey.json"

# Initialize the BigQuery client
client = bigquery.Client.from_service_account_json(service_account_key_path)

# Define the schema for your table
schema = [
    bigquery.SchemaField('code', 'STRING'),
    bigquery.SchemaField('name', 'STRING'),
    bigquery.SchemaField('ForeignInvestorBuyShares', 'INTEGER'),
    bigquery.SchemaField('ForeignInvestorSellShares', 'INTEGER'),
    bigquery.SchemaField('ForeignInvestorNetShares', 'INTEGER'),
    bigquery.SchemaField('InvestmentTrustBuyShares', 'INTEGER'),
    bigquery.SchemaField('InvestmentTrustSellShares', 'INTEGER'),
    bigquery.SchemaField('InvestmentTrustNetShares', 'INTEGER'),
    bigquery.SchemaField('DealerSelfNetTotalShares', 'INTEGER'),
    bigquery.SchemaField('DealerSelfBuyShares', 'INTEGER'),
    bigquery.SchemaField('DealerSelfSellShares', 'INTEGER'),
    bigquery.SchemaField('DealerSelfNetShares', 'INTEGER'),
    bigquery.SchemaField('DealerSelfBuyShareshedge', 'INTEGER'),
    bigquery.SchemaField('DealerSelfSellShareshedge', 'INTEGER'),
    bigquery.SchemaField('DealerSelfNetShareshedge', 'INTEGER'),
    bigquery.SchemaField('TotalCapitalQuantity', 'INTEGER'),
    bigquery.SchemaField('date', 'DATE')
]

# Define the dataset and table ID
dataset_id = 'PythonStock'  # Replace with your dataset ID
table_id = 'DailyStock_Capital'      # Replace with your table ID

# Create a dataset reference
dataset_ref = client.dataset(dataset_id)

# Create the table with the specified schema
table_ref = dataset_ref.table(table_id)
table = bigquery.Table(table_ref, schema=schema)
table = client.create_table(table)  # Create the table

print(f"Table {table_id} created in dataset {dataset_id}.")

In [ ]:
#Insert into BigQuery
job = client.load_table_from_dataframe(data, table_ref)
job.result()  #等待寫入完成

print("Mission Complete!!")

In [ ]:
# For google cloud function
from google.cloud import bigquery
import requests
import pandas as pd
import datetime
import io
from io import StringIO

def CapitalTable(request):

    # Line notify
    def lineNotifyMessage(msg):

            headers = {
            "Authorization": "Bearer " + 'jbScjQe6jY16zTILgQMN1REp6UHeJgCNAXFdJ82P8TZ', 
            "Content-Type" : "application/x-www-form-urlencoded"
            }

            payload = {'message': msg }
            r = requests.post("https://notify-api.line.me/api/notify", headers = headers, params = payload)

    today = datetime.date.today()

    # Format the date as 'yyyymmdd'
    datestr = today.strftime('%Y%m%d')

    # Try if data exists
    try:
        r = requests.get('https://www.twse.com.tw/rwd/zh/fund/T86?date=' + datestr + '&selectType=ALLBUT0999&response=csv')

        # 整理資料，變成表格
        df = pd.read_csv(StringIO(r.text.replace("=", "")), header=1).dropna(how='all', axis=1).dropna(how='any')

        # Remove Column
        df = df.drop(columns='外資自營商買進股數')
        df = df.drop(columns='外資自營商賣出股數')
        df = df.drop(columns='外資自營商買賣超股數')

        field_names = [
        'code',
        'name',
        'ForeignInvestorBuyShares',
        'ForeignInvestorSellShares',
        'ForeignInvestorNetShares',
        'InvestmentTrustBuyShares',
        'InvestmentTrustSellShares',
        'InvestmentTrustNetShares',
        'DealerSelfNetTotalShares',
        'DealerSelfBuyShares',
        'DealerSelfSellShares',
        'DealerSelfNetShares',
        'DealerSelfBuySharesHedge',
        'DealerSelfSellSharesHedge',
        'DealerSelfNetSharesHedge',
        'TotalCapitalQuantity'
        ]

        # Rename the columns of the DataFrame
        df.columns = field_names

        # Turn into numeric
        df[df.columns[2:]] = df[df.columns[2:]].map(lambda x: pd.to_numeric(str(x).replace(",", ""), errors='coerce'))

        # Insert Date
        df['Date'] = datestr

        # Convert the 'Date' column to datetime format
        df['Date'] = pd.to_datetime(df['Date']).dt.date

        ## Get BiqQuery Set up
        client = bigquery.Client()
        dataset_id = 'PythonStock'
        table_id = 'DailyStock_Capital'
        table_ref = client.dataset(dataset_id).table(table_id)

        print(table_ref)
        print(client)

        job = client.load_table_from_dataframe(df, table_ref)
        job.result()  # Wait for completing
        
        lineNotifyMessage("{} Capital Data Inserted".format(datestr))
        return "{} Capital Data Inserted".format(datestr)
        
    except:
        lineNotifyMessage("{} Capital Data Inserted".format(datestr))
        return "{} Capital Data is null".format(datestr)



In [ ]:
# For Temporaly Using
data = CapitalData('20231003')

dataset_id = 'PythonStock'
table_id = 'DailyStock_Capital'
table_ref = client.dataset(dataset_id).table(table_id)

# Insert into BigQuery
job = client.load_table_from_dataframe(data, table_ref)
job.result()  # Wait for completing

# Line notify
def lineNotifyMessage(msg):

        headers = {
        "Authorization": "Bearer " + 'jbScjQe6jY16zTILgQMN1REp6UHeJgCNAXFdJ82P8TZ', 
        "Content-Type" : "application/x-www-form-urlencoded"
        }

        payload = {'message': msg }
        r = requests.post("https://notify-api.line.me/api/notify", headers = headers, params = payload)

lineNotifyMessage("{} Capital Data Inserted".format('20231003'))